In [ ]:

import requests
import urllib.parse
import pandas as pd

properties = requests.get("https://wwwdev.ebi.ac.uk/ols4/api/v2/ontologies/evorao/properties?size=1000").json()['elements']

def get_similar(prop):
    encoded_iri = urllib.parse.quote(urllib.parse.quote(prop['iri'], safe=''), safe='')
    similar = requests.get(f"https://wwwdev.ebi.ac.uk/ols4/api/v2/ontologies/evorao/properties/{encoded_iri}/similar?size=30").json()
    return similar

for prop in properties:
    prop['similar'] = get_similar(prop)

rows = []
for prop in properties:
    for similar in prop['similar']['elements']:
        rows.append({
            'iri': prop['iri'],
            'label': prop['label'][0],
            'similar_label': similar['label'][0],
            'similar_ontology': similar['ontologyId'],
            'score': similar['score'],
            'similar_iri': similar['iri']
        })


df = pd.DataFrame(rows)
df = df.sort_values(by=['iri', 'score'], ascending=[True, False])

df.to_csv('evorao_similar_properties.csv', index=False)



